In [ ]:
%load_ext autoreload
%autoreload 2
import os; print(os.getcwd())
import socket; print(socket.gethostname())

from Bio import SeqIO
import argparse
import json
import os
from pathlib import Path
import random
import sys

In [ ]:
def parse_fasta(fasta_file):
   return SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
parse_fasta('permut2.fasta')

In [30]:
def create_json_structure(job_name, sequences, seeds=None, count=1):
    """Create the JSON structure with separate chains."""
    
    if seeds is None:
        seeds = []
    return {
	"modelSeeds": seeds,
        "name": job_name,
        "sequences": [
            {
                "proteinChain": {
                    "sequence": seq,
                    "count": count
                }
            }
            for i, seq in enumerate(sequences)
        ],
  }
    

def convert_fasta_to_json(fasta_file, seeds=None, count=1, output_dir=None):
    """Convert FASTA files to JSON format.
    Each entry is split into it's own job and chains are split on `:` as in Colabfold.
    Based on code from Julia Varga.
    """
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    
    json_data=[]
    list_of_jobs = parse_fasta(fasta_file)
    for name, seq in list_of_jobs.items():    
        # Create JSON structure

        sequences = str(seq.seq).replace(' ','').split(':')
        name = name.replace('(','_')
        name = name.replace(')','_')
        name = name.replace(';','_')
        job_data = create_json_structure(name, sequences, seeds=seeds, count=count)
        json_data.append(job_data)
        

    # Determine output path    
    input_path = Path(fasta_file)
    
    output_filename = input_path.stem + '.json'
    if output_dir:
        output_path = Path(output_dir) / output_filename
    else:
        output_path = input_path.with_suffix('.json')
    
    # Write JSON file
    with open(output_path, 'w') as f:
        json.dump(json_data, f, indent=2)
        

convert_fasta_to_json(r"D:\OneDrive\Projekti_KI\SIDE\2025-04-01__permuted_dimers\permut2_noCC.fasta")